In [1]:
import os
os.chdir("..")
from contentcreatie.llm_client.prompt_builder import  PromptBuilder
import pandas as pd
from contentcreatie.config.settings import Settings
from contentcreatie.config.paths import paths
from test_notebooks.content_retrieval import llm,doc_store, vector_store
from pipelines.processors import summarize_new_documents, add_new_documents_to_docstore
import sys
settings = Settings()


StorageService: No Connection String found. Attempting DefaultAzureCredential...
/mnt/batch/tasks/shared/LS_root/mounts/clusters/kis-compute-web/code/Users/a.melzer/Contentcreatie
Loaded FAISS index (6723 vectors) and ID set from disk.
Syncing VectorStore with DocumentStore...
VectorStore is already in sync. No new documents to add.


In [2]:
kme_vertaaltabel = pd.read_csv(paths.base_dir /"data" /  "kme_vertaaltabel.csv",sep=';').set_index("KME_ID")
kme_vertaaltabel['BELASTINGSOORT'] = kme_vertaaltabel['BELASTINGSOORT'].str.split('/')

# Opbouwen Doc store van pdf

In [3]:
kme_documents = pd.read_pickle(paths.content_folder / "extracted_df_km.pkl",compression='gzip').reset_index()
kme_documents = kme_documents.set_index("km_nummer").join(kme_vertaaltabel).reset_index()

In [10]:
results = add_new_documents_to_docstore(kme_documents,doc_store)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 189/189 [00:00<00:00, 23289.37it/s]


### Samenvatten en segmenteren content

In [32]:
summary_processor = PromptBuilder(template_path=paths.base_dir/'Contentcreatie'/ 'prompt_templates',name='summarize')

In [33]:
import logging
import sys

# Set up logging to see all debug messages from the Azure SDK
logger = logging.getLogger('azure')
logger.setLevel(logging.DEBUG)

# Create a handler to write logs to standard out
handler = logging.StreamHandler(stream=sys.stdout)
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)

In [34]:
stats = summarize_new_documents(
    doc_store=doc_store,
    prompt_builder=summary_processor,
    llm=llm,
    max_workers=20,
    start=0,
    count=7000,
    show_progress=True
)

In [18]:
doc_store.save()
doc_store.rebuild_search_index()

Rebuilding search index for 6723 documents...
Successfully indexed 6723 documents.


In [30]:
from dataclasses import dataclass, asdict
import json

def save(dstore):
        """Saves the current document dictionary to a Parquet file."""
        if not dstore.documents:
            # Save an empty DataFrame with the correct schema
            df = pd.DataFrame(columns=['id', 'title', 'content', 'metadata'])
        else:
            # Convert dict of Document objects to a list of dicts
            doc_list = [asdict(doc) for doc in dstore.documents.values()]
            df = pd.DataFrame.from_records(doc_list)
            
            # Serialize the metadata dict to a JSON string for Parquet compatibility
            df['metadata'] = df['metadata'].apply(json.dumps)
        persist= '/home/azureuser/cloudfiles/code/Users/a.melzer/Contentcreatie/data/docstores/kme_content/documents.parquet'
        try:
            # Save to Parquet using pyarrow engine
            df.to_parquet(persist, index=False, engine='pyarrow')
        except Exception as e:
            print(f"Error saving Parquet file {persist}: {e}")

In [20]:
save(doc_store)

### Update taxonomie

In [12]:
docs = kme_documents.set_index('km_nummer')
for doc in doc_store.documents:
    if(len(doc_store.documents[doc].metadata['BELASTINGSOORT'])>1):
        print(1)
    
    

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [31]:
save(doc_store)